<a href="https://colab.research.google.com/github/viswan29/Machine-Learning/blob/master/Copy_of_CourseRecom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import turicreate
import pandas as pd
import numpy as np

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
users_lex = pd.read_excel('Users_LEX.xlsx')
Ratings_lex = pd.read_excel('Emp_Ratings.xlsx')
lex_courses = pd.read_excel('LEX_ITEMS.xlsx')
ratings_test = pd.read_excel('Rating_test.xlsx')
ratings_train = pd.read_excel('Rating_train1.xlsx')

FileNotFoundError: ignored

In [0]:
ratings_train.columns

Index(['user_id', 'course_id', 'Rating'], dtype='object')

In [0]:
n_users = Ratings_lex['user_id'].unique().shape[0]
n_courses = Ratings_lex['course_id'].unique().shape[0]

In [0]:
data_matrix = np.zeros((n_users, n_courses))
for line in Ratings_lex.itertuples():
    data_matrix[line[1]-1, line[2]-1] = line[3]

In [0]:
from sklearn.metrics.pairwise import pairwise_distances 
user_similarity = pairwise_distances(data_matrix, metric='cosine')
course_similarity = pairwise_distances(data_matrix.T, metric='cosine')

In [0]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'course':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [0]:
user_prediction = predict(data_matrix, user_similarity, type='user')
course_prediction = predict(data_matrix, course_similarity, type='course')

In [0]:
test_data = turicreate.SFrame(ratings_test)
train_data = turicreate.SFrame(ratings_train)

In [0]:
popularity_model = turicreate.popularity_recommender.create(train_data, user_id='user_id', item_id='course_id', target='Rating')

Preparing data set.

Data has 312 observations with 118 users and 29 items.

Data prepared in: 0.011984s

312 observations to process; with 29 unique items.

In [0]:
popularity_recomm = popularity_model.recommend(users=[1,2,3,4,5],k=2)
popularity_recomm.print_rows(num_rows=25)

+---------+-----------+--------------------+------+
| user_id | course_id |       score        | rank |
+---------+-----------+--------------------+------+
|    1    |     15    | 3.6923076923076925 |  1   |
|    1    |     13    | 3.5714285714285716 |  2   |
|    2    |     13    | 3.5714285714285716 |  1   |
|    2    |     8     |       3.375        |  2   |
|    3    |     15    | 3.6923076923076925 |  1   |
|    3    |     13    | 3.5714285714285716 |  2   |
|    4    |     15    | 3.6923076923076925 |  1   |
|    4    |     13    | 3.5714285714285716 |  2   |
|    5    |     13    | 3.5714285714285716 |  1   |
|    5    |     8     |       3.375        |  2   |
+---------+-----------+--------------------+------+
[10 rows x 4 columns]



In [0]:
#Training the model
item_sim_model = turicreate.item_similarity_recommender.create(train_data, user_id='user_id', item_id='course_id', target='Rating', similarity_type='cosine')

#Making recommendations
item_sim_recomm = item_sim_model.recommend(users=[1,2,3,4,5],k=2)
item_sim_recomm.print_rows(num_rows=25)

Preparing data set.

Data has 312 observations with 118 users and 29 items.

Data prepared in: 0.011295s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.014327s

+---------+-----------+---------------------+------+
| user_id | course_id |        score        | rank |
+---------+-----------+---------------------+------+
|    1    |     3     |  0.4496081471443176 |  1   |
|    1    |     13    |  0.3814370036125183 |  2   |
|    2    |     5     |  0.6481551130612692 |  1   |
|    2    |     9     |  0.6351969838142395 |  2   |
|    3    |     5     |  0.519952118396759  |  1   |
|    3    |     6     |  0.3620702922344208 |  2   |
|    4    |     3     |  0.6584172248840332 |  1   |
|    4    |     29    | 0.38849806785583496 |  2   |
|    5    |     4     |  0.5294990638891856 |  1   |
|    5    |     19    |  0.4389215310414632 |  2   |
+---------+-----------+---------------------+------+
[10 rows x 4 columns]

